In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import datasets as ds


## The dataset

### Question 1
How many splits does the dataset has?

In [6]:
splits = ds.get_dataset_split_names('imdb')
print(splits)
print(f'Number of splits: {len(splits)}')

['train', 'test', 'unsupervised']
Number of splits: 3


There are 3 splits in the IMDB dataset.

### Question 2
How big are these splits?

In [7]:
dataset_0 = ds.load_dataset('imdb', split=splits[0])
dataset_1 = ds.load_dataset('imdb', split=splits[1])
dataset_2 = ds.load_dataset('imdb', split=splits[2])

Found cached dataset imdb (/home/bastien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/home/bastien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/home/bastien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [8]:
print(f'{splits[0]} split size : {dataset_0.num_rows}')
print(f'{splits[1]} split size : {dataset_1.num_rows}')
print(f'{splits[2]} split size : {dataset_2.num_rows}')

train split size : 25000
test split size : 25000
unsupervised split size : 50000


### Question 3
What is the proportion of each class on the supervised splits?

In [56]:
# Dataset 0: Train
train_data_frame = dataset_0.to_pandas()
print("Train")
print(train_data_frame.where(train_data_frame['label'] == 0).count())
print(train_data_frame.where(train_data_frame['label'] == 1).count())

print("\n")
# Dataset 1: Test
print("Test")
test_data_frame = dataset_1.to_pandas()
print(test_data_frame.where(test_data_frame['label'] == 0).count())
print(test_data_frame.where(test_data_frame['label'] == 1).count())


Train
text     12500
label    12500
dtype: int64
text     12500
label    12500
dtype: int64


Test
text     12500
label    12500
dtype: int64
text     12500
label    12500
dtype: int64


Hence, each class represents 50% of the supervised dataset (both in train and test samples).

## Naive Bayes classifier 

### Question 1
Create an adapted processing function which lower case the text and replace punctuations with text:

In [54]:
import string
import re
from bs4 import BeautifulSoup

def replace_punctuation(text: str, replace: str)-> str:
  return re.sub('[%s]' % re.escape(string.punctuation), replace, text)

def remove_multiple_space(text : str) -> str:
  return re.sub(' +', ' ', text)

def clean_html(text: str) -> str:
  no_html = BeautifulSoup(text).get_text()
  clean = re.sub("[^a-z\s]+", " ", no_html, flags=re.IGNORECASE)
  return re.sub("(\s+)", " ", clean)

def text_processing(text: str) -> str:
  result_text = text
    #result_text = replace_punctuation(text,' ') 
    #
    #result_text = result_text.strip()
    #result_text = remove_multiple_space(result_text)
  result_text = clean_html(result_text)
  result_text = result_text.lower()
  return result_text

#tiny test
#assert text_processing("Hello, ,,,World!::") ==  'hello world'

In [57]:
train_data_frame.text[4]

'Oh, brother...after hearing about this ridiculous film for umpteen years all I can think of is that old Peggy Lee song..<br /><br />"Is that all there is??" ...I was just an early teen when this smoked fish hit the U.S. I was too young to get in the theater (although I did manage to sneak into "Goodbye Columbus"). Then a screening at a local film museum beckoned - Finally I could see this film, except now I was as old as my parents were when they schlepped to see it!!<br /><br />The ONLY reason this film was not condemned to the anonymous sands of time was because of the obscenity case sparked by its U.S. release. MILLIONS of people flocked to this stinker, thinking they were going to see a sex film...Instead, they got lots of closeups of gnarly, repulsive Swedes, on-street interviews in bland shopping malls, asinie political pretension...and feeble who-cares simulated sex scenes with saggy, pale actors.<br /><br />Cultural icon, holy grail, historic artifact..whatever this thing was,

Now let's apply `text_processing` function on our dataframe.

In [58]:
train_data_frame.text = train_data_frame.text.apply(text_processing)

In [59]:
train_data_frame.text[4]

'Oh brother after hearing about this ridiculous film for umpteen years all I can think of is that old Peggy Lee song Is that all there is I was just an early teen when this smoked fish hit the U S I was too young to get in the theater although I did manage to sneak into Goodbye Columbus Then a screening at a local film museum beckoned Finally I could see this film except now I was as old as my parents were when they schlepped to see it The ONLY reason this film was not condemned to the anonymous sands of time was because of the obscenity case sparked by its U S release MILLIONS of people flocked to this stinker thinking they were going to see a sex film Instead they got lots of closeups of gnarly repulsive Swedes on street interviews in bland shopping malls asinie political pretension and feeble who cares simulated sex scenes with saggy pale actors Cultural icon holy grail historic artifact whatever this thing was shred it burn it then stuff the ashes in a lead box Elite esthetes still